In [3]:
import os
import getpass

LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="KEY
LANGSMITH_PROJECT="testingToolsAgents"
OPENAI_API_KEY="KEY"

os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [4]:
pip install --upgrade --quiet  langchain-community langchainhub langgraph


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --quiet psycopg2



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import sqlite3
import requests
from sqlalchemy import create_engine
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy.pool import StaticPool


def get_engine_for_postgresql_db():
    """
    Connect to a local PostgreSQL database and create an engine.
    """
    # Define your PostgreSQL connection details
    db_user = "amanaman"        
    db_password = "amansoni"   
    db_host = "localhost"            
    db_port = "5432"                
    db_name = "example_db"  

    # Create the engine for PostgreSQL
    connection_url = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
    return create_engine(connection_url)


# Create the engine
engine = get_engine_for_postgresql_db()

# Create the SQLDatabase instance
db = SQLDatabase(engine)

# Example usage (optional)
if __name__ == "__main__":
    # Test the connection by running a query
    try:
        result = db.run("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
        print("Tables in the database:", result)
    except Exception as e:
        print("Error connecting to the database:", e)


Tables in the database: [('department',), ('project',), ('employee',), ('task',), ('employee_task',)]


In [7]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [12]:
# tools overview.
print(toolkit.get_tools())

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x1074787d0>), InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x1074787d0>), ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x1074787d0>), QuerySQLCheckerTool(description='Use this tool to double check if your que

In [13]:
# using the individual tools directly:
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDatabaseTool,
)

### Use within an agent
we equip a simple question-answering agent with the tools in our toolkit

In [14]:
from langchain import hub
prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

['dialect', 'top_k']


In [15]:
system_message = prompt_template.format(dialect="SQLite", top_k=5)

In [16]:
# agent innitiation
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(
    llm, toolkit.get_tools(), state_modifier=system_message
)

In [17]:
# Asking a query
example_query = "Which employee have the most salary"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Which employee have the most salary
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_PGI7xDGw2sjTd6UeLaRwVKiB)
 Call ID: call_PGI7xDGw2sjTd6UeLaRwVKiB
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

department, employee, employee_task, project, task
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_KzWFxKIljKU8JssCZHvzcy6e)
 Call ID: call_KzWFxKIljKU8JssCZHvzcy6e
  Args:
    table_names: employee
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE employee (
	employee_id SERIAL NOT NULL, 
	name VARCHAR(255) NOT NULL, 
	age INTEGER NOT NULL, 
	salary NUMERIC(10, 2) NOT NULL, 
	department_id INTEGER, 
	CONSTRAINT employee_pkey PRIMARY KEY (emplo

In [18]:
# If error occurs?
example_query = "Who are the top 3 best selling artists?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Who are the top 3 best selling artists?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_3LhKrwGVcCEIq5jz8mgBAnW3)
 Call ID: call_3LhKrwGVcCEIq5jz8mgBAnW3
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

department, employee, employee_task, project, task
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_vN7xYBkxru2eD7KPhaq4zDHd)
 Call ID: call_vN7xYBkxru2eD7KPhaq4zDHd
  Args:
    table_names: department
  sql_db_schema (call_LhYdudPU4Ym9mx6cftndOoqO)
 Call ID: call_LhYdudPU4Ym9mx6cftndOoqO
  Args:
    table_names: employee
  sql_db_schema (call_xhQSRRS2UbgKI6Bjt8hDveH8)
 Call ID: call_xhQSRRS2UbgKI6Bjt8hDveH8
  Args:
    table_names: employee_task
  sql_db_schema (call_CfKwHeLHzQ7CLrY6V4LqBpPE)
 Call ID: ca

### ---Testing wrong questions----

In [21]:
# Asking a query
example_query = "what was the last question I asked?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

what was the last question I asked?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_eepT1cgK0Q5g5rVMR6dGH659)
 Call ID: call_eepT1cgK0Q5g5rVMR6dGH659
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

department, employee, employee_task, project, task
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_LUoQ99LIyY9OfiXkUHAo2HXZ)
 Call ID: call_LUoQ99LIyY9OfiXkUHAo2HXZ
  Args:
    table_names: department
  sql_db_schema (call_gENqwpQhsXfOuqxRpTCK3CEc)
 Call ID: call_gENqwpQhsXfOuqxRpTCK3CEc
  Args:
    table_names: employee
  sql_db_schema (call_zlTnP0MawG3C2hCzPGkQBrpq)
 Call ID: call_zlTnP0MawG3C2hCzPGkQBrpq
  Args:
    table_names: employee_task
  sql_db_schema (call_yy9T4t73C0NC4d0Fn0fQOeDY)
 Call ID: call_y

## SUMUP

In [53]:
import os
import getpass

LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="KEY"
LANGSMITH_PROJECT="testingToolsAgents"
OPENAI_API_KEY="KEY"

os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

import sqlite3
import requests
from sqlalchemy import create_engine
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy.pool import StaticPool
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDatabaseTool,
)
from langchain import hub
from langgraph.prebuilt import create_react_agent


def get_engine_for_postgresql_db():
    """
    Connect to a local PostgreSQL database and create an engine.
    """
    # Define your PostgreSQL connection details
    db_user = "amanaman"        
    db_password = "amansoni"   
    db_host = "localhost"            
    db_port = "5432"                
    db_name = "example_db"  

    # Create the engine for PostgreSQL
    connection_url = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
    return create_engine(connection_url)


# Create the engine
engine = get_engine_for_postgresql_db()

# Create the SQLDatabase instance
db = SQLDatabase(engine)

# Example usage (optional)
if __name__ == "__main__":
    # Test the connection by running a query
    try:
        result = db.run("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
        print("Tables in the database:", result)
    except Exception as e:
        print("Error connecting to the database:", e)



llm = ChatOpenAI(model="gpt-4o-mini")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)


prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

system_message = prompt_template.format(dialect="SQLite", top_k=5)
# agent innitiation


agent_executor = create_react_agent(
    llm, toolkit.get_tools(), state_modifier=system_message
)

# Asking a query
example_query = "which employee have the minimum salary?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

Tables in the database: [('department',), ('project',), ('employee',), ('task',), ('employee_task',)]
['dialect', 'top_k']
================================ Human Message =================================

which employee have the minimum salary?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_1u7i3cIjUCdVaqjtO2Xdr7CS)
 Call ID: call_1u7i3cIjUCdVaqjtO2Xdr7CS
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

department, employee, employee_task, project, task
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_iMrUyP7rWbgoewXG1jnN8Re1)
 Call ID: call_iMrUyP7rWbgoewXG1jnN8Re1
  Args:
    table_names: employee
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE employee (
	employee_id SERIAL NOT NULL, 
	name VARCHAR(255) NOT NULL, 


# Memory intgration in Agents

In [56]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
message_history = ChatMessageHistory()

import os
import getpass

LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="KEY"
LANGSMITH_PROJECT="testingToolsAgents"
OPENAI_API_KEY="KEY"

os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

import sqlite3
import requests
from sqlalchemy import create_engine
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy.pool import StaticPool
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDatabaseTool,
)
from langchain import hub
from langgraph.prebuilt import create_react_agent


def get_engine_for_postgresql_db():
    """
    Connect to a local PostgreSQL database and create an engine.
    """
    # Define your PostgreSQL connection details
    db_user = "amanaman"        
    db_password = "amansoni"   
    db_host = "localhost"            
    db_port = "5432"                
    db_name = "example_db"  

    # Create the engine for PostgreSQL
    connection_url = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
    return create_engine(connection_url)


# Create the engine
engine = get_engine_for_postgresql_db()

# Create the SQLDatabase instance
db = SQLDatabase(engine)

# Example usage (optional)
if __name__ == "__main__":
    # Test the connection by running a query
    try:
        result = db.run("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
        print("Tables in the database:", result)
    except Exception as e:
        print("Error connecting to the database:", e)



llm = ChatOpenAI(model="gpt-4o-mini")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)


prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

system_message = prompt_template.format(dialect="SQLite", top_k=5)
# agent innitiation
from langchain_core.messages import AIMessage, HumanMessage







#actual
agent_executor = create_react_agent(
    llm, toolkit.get_tools(), state_modifier=system_message,
)

# agent_executor.invoke(
#     {
#         "chat_history": [
#             HumanMessage(content="hi! my name is bob"),
#             AIMessage(content="Hello Bob! How can I assist you today?"),
#         ],
#         "input": "Hi, I want you to answer some questions",
#     }
# )


agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Asking a query
example_query = "what is the relation between the employee and department?"

events = agent_with_chat_history.invoke(
    {"messages": [("user", example_query)]},
    stream_mode="values",
    config={"configurable": {"session_id": "<foo>"}}
)
# for event in events:
#     event["messages"][-1].pretty_print()

Tables in the database: [('department',), ('project',), ('employee',), ('task',), ('employee_task',)]
['dialect', 'top_k']


Error in RootListenersTracer.on_chain_end callback: KeyError('input')


In [50]:
for i in events:
    print(i)


messages


In [65]:
# events["messages"][-1].pretty_print()
events['messages'][-1]

AIMessage(content="The relationship between the `employee` and `department` tables is established through a foreign key constraint. Here are the key points:\n\n- The `employee` table contains a column named `department_id`, which references the `department_id` in the `department` table.\n- This means that each employee is associated with a specific department, identified by the department's ID.\n- The `department` table includes `department_id` as its primary key.\n\nIn summary, each employee can belong to one department, while each department can have multiple employees associated with it, establishing a one-to-many relationship between departments and employees.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 125, 'prompt_tokens': 884, 'total_tokens': 1009, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0,